In [1]:
import keras
import keras.backend as K
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.callbacks import CSVLogger
import keras.utils
import os
import keras.activations
import time
import numpy as np

C:\Users\MaksutovRN\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [3]:
def normalize_sample_wise(dataset):
    normalized = []
    for s in dataset:
        mn, mx = s.min(), s.max()
        s = (s - mn) / (mx - mn)
        normalized.append(s)
    return np.array(normalized)

x_train = normalize_sample_wise(x_train)
x_test = normalize_sample_wise(x_test)

In [4]:
batch_size = 64
epochs = 100
units = 64
experiments = 5
start = 0
# activations = ['sigmoid', 'tanh', 'relu', 'linear', 'elu', 'softplus', 'softsign', 'hard_sigmoid', 'LeakyReLU', 'ThresholdedReLU']
activations = ['relu']

In [5]:
start_time = time.time()
counter = 0
total_items = len(activations) * experiments

for act in activations:
    print("Training for activation " + act + " with RMSProp optimizer")

    for i in range(experiments):

        K.clear_session()
        K.reset_uids()

        act_dict = {
          'sigmoid': Activation(keras.activations.sigmoid),
          'tanh': Activation(keras.activations.tanh),
          'relu': Activation(keras.activations.relu),
          'linear': Activation(keras.activations.linear),
          'elu': Activation(keras.activations.elu),
          'softplus': Activation(keras.activations.softplus),
          'softsign': Activation(keras.activations.softsign),
          'hard_sigmoid': Activation(keras.activations.hard_sigmoid),
          'LeakyReLU': keras.layers.advanced_activations.LeakyReLU(),
          'selu': Activation(keras.activations.selu),
          'ThresholdedReLU': keras.layers.advanced_activations.ThresholdedReLU(theta=0.7) 
        }

        model_name = 'normd_sample_wise_' + act + '_rmsp_' + str(i + start) + '_' + str(units)
        inputs = Input(shape=(784,))
        x = Dense(units)(inputs)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        x = Dense(units)(x)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        predictions = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=inputs, outputs=predictions)

        model.compile(loss='categorical_crossentropy',
                          optimizer=RMSprop(lr=0.001),
                          metrics=['accuracy'])

        csv_logger = CSVLogger('./training_logs/%s.csv' % (model_name), append=False)
        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(x_test, y_test), callbacks=[csv_logger])

        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        t = time.time()
        time_diff = t - start_time
        counter +=1
        rem_items = total_items - counter
        total_time = round((total_items / counter) * time_diff)
        rem_time = round(total_time - time_diff)
        m, s = divmod(rem_time, 60)
        h, m = divmod(m, 60)
        d, h = divmod(h, 24)
        print('Remaining time: %d days %d hours %02d minutes %02d seconds' % (d, h, m, s))

Training for activation relu with RMSProp optimizer
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 6s 107us/step - loss: 0.4453 - acc: 0.8685 - val_loss: 0.1781 - val_acc: 0.9475
Epoch 2/100
60000/60000 [==============================] - 4s 68us/step - loss: 0.2282 - acc: 0.9330 - val_loss: 0.1354 - val_acc: 0.9596
Epoch 3/100
60000/60000 [==============================] - 4s 66us/step - loss: 0.1854 - acc: 0.9468 - val_loss: 0.1210 - val_acc: 0.9648
Epoch 4/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1644 - acc: 0.9526 - val_loss: 0.1224 - val_acc: 0.9637
Epoch 5/100
60000/60000 [==============================] - 4s 66us/step - loss: 0.1499 - acc: 0.9574 - val_loss: 0.1132 - val_acc: 0.9694
Epoch 6/100
60000/60000 [==============================] - 4s 66us/step - loss: 0.1415 - acc: 0.9604 - val_loss: 0.1091 - val_acc: 0.9725
Epoch 7/100
60000/60000 [==============================] - 4s 66us/st

60000/60000 [==============================] - 3s 58us/step - loss: 0.1131 - acc: 0.9751 - val_loss: 0.1680 - val_acc: 0.9737
Epoch 60/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.1089 - acc: 0.9761 - val_loss: 0.1611 - val_acc: 0.9748
Epoch 61/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.1174 - acc: 0.9758 - val_loss: 0.1670 - val_acc: 0.9742
Epoch 62/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1107 - acc: 0.9754 - val_loss: 0.1738 - val_acc: 0.9753
Epoch 63/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.1135 - acc: 0.9762 - val_loss: 0.1673 - val_acc: 0.9743
Epoch 64/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.1139 - acc: 0.9755 - val_loss: 0.1764 - val_acc: 0.9747
Epoch 65/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1169 - acc: 0.9764 - val_loss: 0.1707 - val_acc: 0.9750
Epoch 66/100
60000/60000 [==============

Epoch 17/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.1225 - acc: 0.9686 - val_loss: 0.1134 - val_acc: 0.9762
Epoch 18/100
60000/60000 [==============================] - 4s 65us/step - loss: 0.1188 - acc: 0.9697 - val_loss: 0.1204 - val_acc: 0.9757
Epoch 19/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.1174 - acc: 0.9691 - val_loss: 0.1174 - val_acc: 0.9767
Epoch 20/100
60000/60000 [==============================] - 4s 65us/step - loss: 0.1133 - acc: 0.9708 - val_loss: 0.1188 - val_acc: 0.9746
Epoch 21/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.1177 - acc: 0.9714 - val_loss: 0.1211 - val_acc: 0.9760
Epoch 22/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1116 - acc: 0.9716 - val_loss: 0.1276 - val_acc: 0.9759
Epoch 23/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1191 - acc: 0.9711 - val_loss: 0.1192 - val_acc: 0.9752
Epoch 24/100
60000/60000 [=

Epoch 76/100
60000/60000 [==============================] - 4s 68us/step - loss: 0.1256 - acc: 0.9760 - val_loss: 0.2012 - val_acc: 0.9733
Epoch 77/100
60000/60000 [==============================] - 4s 70us/step - loss: 0.1240 - acc: 0.9758 - val_loss: 0.1912 - val_acc: 0.9740
Epoch 78/100
60000/60000 [==============================] - 4s 64us/step - loss: 0.1265 - acc: 0.9753 - val_loss: 0.1862 - val_acc: 0.9761
Epoch 79/100
60000/60000 [==============================] - 4s 66us/step - loss: 0.1260 - acc: 0.9766 - val_loss: 0.1838 - val_acc: 0.9761
Epoch 80/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.1281 - acc: 0.9756 - val_loss: 0.1988 - val_acc: 0.9743
Epoch 81/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1297 - acc: 0.9757 - val_loss: 0.1967 - val_acc: 0.9739
Epoch 82/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1231 - acc: 0.9765 - val_loss: 0.1914 - val_acc: 0.9740
Epoch 83/100
60000/60000 [=

Epoch 34/100
60000/60000 [==============================] - 4s 68us/step - loss: 0.1077 - acc: 0.9747 - val_loss: 0.1382 - val_acc: 0.9777
Epoch 35/100
60000/60000 [==============================] - 4s 70us/step - loss: 0.1096 - acc: 0.9737 - val_loss: 0.1386 - val_acc: 0.9768
Epoch 36/100
60000/60000 [==============================] - 4s 72us/step - loss: 0.1063 - acc: 0.9749 - val_loss: 0.1416 - val_acc: 0.9758
Epoch 37/100
60000/60000 [==============================] - 4s 71us/step - loss: 0.1074 - acc: 0.9754 - val_loss: 0.1365 - val_acc: 0.9774
Epoch 38/100
60000/60000 [==============================] - 4s 73us/step - loss: 0.1059 - acc: 0.9751 - val_loss: 0.1428 - val_acc: 0.9770
Epoch 39/100
60000/60000 [==============================] - 4s 66us/step - loss: 0.1081 - acc: 0.9756 - val_loss: 0.1422 - val_acc: 0.9778
Epoch 40/100
60000/60000 [==============================] - 4s 65us/step - loss: 0.1059 - acc: 0.9753 - val_loss: 0.1436 - val_acc: 0.9766
Epoch 41/100
60000/60000 [=

Epoch 93/100
60000/60000 [==============================] - 4s 66us/step - loss: 0.1216 - acc: 0.9777 - val_loss: 0.1779 - val_acc: 0.9746
Epoch 94/100
60000/60000 [==============================] - 4s 65us/step - loss: 0.1182 - acc: 0.9773 - val_loss: 0.1874 - val_acc: 0.9738
Epoch 95/100
60000/60000 [==============================] - 4s 65us/step - loss: 0.1185 - acc: 0.9778 - val_loss: 0.1941 - val_acc: 0.9751
Epoch 96/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1249 - acc: 0.9770 - val_loss: 0.1877 - val_acc: 0.9751
Epoch 97/100
60000/60000 [==============================] - 4s 64us/step - loss: 0.1239 - acc: 0.9770 - val_loss: 0.1994 - val_acc: 0.9738
Epoch 98/100
60000/60000 [==============================] - 4s 65us/step - loss: 0.1259 - acc: 0.9769 - val_loss: 0.1955 - val_acc: 0.9746
Epoch 99/100
60000/60000 [==============================] - 4s 68us/step - loss: 0.1199 - acc: 0.9781 - val_loss: 0.1899 - val_acc: 0.9744
Epoch 100/100
60000/60000 [

Epoch 51/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.1110 - acc: 0.9746 - val_loss: 0.1467 - val_acc: 0.9750
Epoch 52/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.1134 - acc: 0.9756 - val_loss: 0.1504 - val_acc: 0.9738
Epoch 53/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.1128 - acc: 0.9744 - val_loss: 0.1509 - val_acc: 0.9761
Epoch 54/100
60000/60000 [==============================] - 4s 63us/step - loss: 0.1138 - acc: 0.9748 - val_loss: 0.1557 - val_acc: 0.9762
Epoch 55/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.1169 - acc: 0.9747 - val_loss: 0.1553 - val_acc: 0.9744
Epoch 56/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.1124 - acc: 0.9755 - val_loss: 0.1540 - val_acc: 0.9752
Epoch 57/100
60000/60000 [==============================] - 4s 65us/step - loss: 0.1188 - acc: 0.9740 - val_loss: 0.1491 - val_acc: 0.9740
Epoch 58/100
60000/60000 [=

Epoch 9/100
60000/60000 [==============================] - 4s 66us/step - loss: 0.1279 - acc: 0.9642 - val_loss: 0.1136 - val_acc: 0.9734
Epoch 10/100
60000/60000 [==============================] - 4s 67us/step - loss: 0.1259 - acc: 0.9653 - val_loss: 0.1094 - val_acc: 0.9708
Epoch 11/100
60000/60000 [==============================] - 4s 67us/step - loss: 0.1236 - acc: 0.9658 - val_loss: 0.1163 - val_acc: 0.9722
Epoch 12/100
60000/60000 [==============================] - 4s 71us/step - loss: 0.1205 - acc: 0.9678 - val_loss: 0.1136 - val_acc: 0.9734
Epoch 13/100
60000/60000 [==============================] - 4s 71us/step - loss: 0.1172 - acc: 0.9679 - val_loss: 0.1105 - val_acc: 0.9733
Epoch 14/100
60000/60000 [==============================] - 4s 68us/step - loss: 0.1185 - acc: 0.9668 - val_loss: 0.1184 - val_acc: 0.9730
Epoch 15/100
60000/60000 [==============================] - 4s 67us/step - loss: 0.1162 - acc: 0.9688 - val_loss: 0.1092 - val_acc: 0.9743
Epoch 16/100
60000/60000 [==

Epoch 68/100
60000/60000 [==============================] - 4s 64us/step - loss: 0.1117 - acc: 0.9769 - val_loss: 0.1768 - val_acc: 0.9738
Epoch 69/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1143 - acc: 0.9760 - val_loss: 0.1790 - val_acc: 0.9742
Epoch 70/100
60000/60000 [==============================] - 4s 65us/step - loss: 0.1082 - acc: 0.9771 - val_loss: 0.1813 - val_acc: 0.9725
Epoch 71/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.1132 - acc: 0.9762 - val_loss: 0.1801 - val_acc: 0.9730
Epoch 72/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1108 - acc: 0.9766 - val_loss: 0.1723 - val_acc: 0.9738
Epoch 73/100
60000/60000 [==============================] - 4s 65us/step - loss: 0.1158 - acc: 0.9764 - val_loss: 0.1657 - val_acc: 0.9743
Epoch 74/100
60000/60000 [==============================] - 4s 72us/step - loss: 0.1127 - acc: 0.9768 - val_loss: 0.1764 - val_acc: 0.9742
Epoch 75/100
60000/60000 [=